In [ ]:
!pip3 install thirdai --upgrade
!pip3 install thirdai[neural_db]
!pip3 install langchain --upgrade
!pip3 install openai --upgrade
!pip3 install paper-qa --upgrade
!pip3 install gradio
!pip3 install json
!pip3 install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 58.3 MB/s eta 0:00:0

In [ ]:
import thirdai
thirdai.licensing.activate("ABC")

In [ ]:
from thirdai import neural_db as ndb
db = ndb.NeuralDB(user_id="my_user")

In [ ]:
# Set up a cache directory
import os
if not os.path.isdir("bazaar_cache"):
    os.mkdir("bazaar_cache")

from pathlib import Path
from thirdai.neural_db import Bazaar
bazaar = Bazaar(cache_dir=Path("bazaar_cache"))
bazaar.fetch()

In [ ]:
print(bazaar.list_model_names())


['Finance QnA', 'General QnA', 'Contract Review']


In [ ]:
db = bazaar.get_model("General QnA")

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
insertable_docs = []
pdf_files = ['Face recognition.pdf']
for file in pdf_files:
    pdf_doc = ndb.PDF(file)
    insertable_docs.append(pdf_doc)

In [ ]:
source_ids = db.insert(insertable_docs, train=True)

loaded data | source 'Documents:
Face recognition.pdf' | vectors 40 | batches 1 | time 0s | complete

train | epoch 0 | train_steps 2461 | train_hash_precision@5=0.685  | train_batches 1 | time 0s

train | epoch 1 | train_steps 2462 | train_hash_precision@5=0.96  | train_batches 1 | time 0s

train | epoch 2 | train_steps 2463 | train_hash_precision@5=1  | train_batches 1 | time 0s

train | epoch 3 | train_steps 2464 | train_hash_precision@5=1  | train_batches 1 | time 0s

train | epoch 4 | train_steps 2465 | train_hash_precision@5=1  | train_batches 1 | time 0s

train | epoch 5 | train_steps 2466 | train_hash_precision@5=1  | train_batches 1 | time 0s



In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "ABC"

In [ ]:
from langchain.chat_models import ChatOpenAI
from paperqa.prompts import qa_prompt
from paperqa.chains import make_chain

llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.1,
    max_tokens=150
)


qa_chain = make_chain(prompt=qa_prompt, llm=llm)



In [ ]:

feynman_technique_prompt = """
Instructions:
- Break down the entire thing to parts
- Explain each step in detail.
- Provide examples of how the technique can be used to learn new concepts.
- Evaluate at the end of explanation and explain again as necessary

Constraints:
- Your explanation should be clear, concise, and easy to understand.
- Your explanation should be accurate and factually correct.
- Your explanation should be creative and engaging.


Asking Questions to evaluate learning is a must
"""


In [ ]:
def get_references(query):
    search_results = db.search(query,top_k=3)
    references = []
    for result in search_results:
        references.append(result.text)
    return references

def get_answer(query, references):
    return qa_chain.run(question=query, context='\n\n'.join(references[:3]), answer_length="abt 50 words")

In [ ]:
query0 = "Explain face recognition through eigen values"
# Combine the prompt and the user query
query =  "\n" + f"Query: {query0}\nResponse:" + feynman_technique_prompt

references = get_references(query0)
print(references)

['2 Problem Statement Design a real-time face recognition system using eigenfaces. 3 Steps 1. Understand the algorithm for face recognition described in the following papers: * https://ieeexplore.ieee.org/abstract/document/6793549 [PDF] * https://ieeexplore.ieee.org/document/139758 [PDF] 2. Implement the real-time face recognition algorithms used in the papers on one of the face datasets present here: https://www.face-rec.org/databases/ 3. Prepare and evaluate on a dataset with at least 20 diverse faces (each face having numerous images with some variation in expression lighting etc). Also choose the test size appropriately. 4. Testing the algorithm on a new image should give whether the face is present in the dataset and if yes how many matches for that face are present. Report the accuracy of the evaluation.', 'Assignment 2: Eigenfaces for Face Recognition Last date to submit: 16 October 2022 1 Background The primary role in conveying identity and emotion in human interactions is of 

In [ ]:
answer = get_answer(query, references)
print(answer)

RateLimitError: ignored

In [ ]:
!pip install gTTS pydub

from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play

text = answer
tts = gTTS(text)
tts.save("output.mp3")
# audio = AudioSegment.from_mp3("output.mp3")
# play(audio)



Interface(still in works)

In [ ]:
import gradio as gr
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play

def get_references(query):
    # Implementation for getting references
    pass

def get_answer(query, references):
    # Implementation for getting answer from references
    pass

def generate_audio(query):
    audio = AudioSegment.from_mp3("output.mp3")
    play(audio)

def generate_audio_callback(query):
    generate_audio(query)
    return get_answer(query, get_references(query))

iface = gr.Interface(
    fn=generate_audio_callback,
    inputs="text",
    outputs="text",
    title="Question Answering System",
    description="Enter your query and click 'Generate Audio' to listen to the answer.",
    examples=[["Click 'Generate Audio' to listen to the answer."]]
)

iface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b4b679ff-22af-4d5b-93ed-efe163b8c376' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>